In [ ]:
import xarray as xr
import numpy as np
import his_utils
import pandas as pd
import os
from concurrent.futures import ProcessPoolExecutor, as_completed

input_data = xr.open_dataset('testdata/2021-06-26/ERA5_input.nc').squeeze('batch')

In [5]:
perturbations = [0.001, 0.1, 1, 2, 3, 4, 5]


for scale in perturbations:
    new_data = his_utils.add_perturbation(input_data,
                                            ['2m_temperature',
                                             '10m_u_component_of_wind',
                                             '10m_v_component_of_wind',
                                             'mean_sea_level_pressure'], 
                                             scale)
    new_data = new_data.expand_dims(dim='batch', axis=0)
    new_data.to_netcdf(f'testdata/2021-06-26/ERA5_4var_{scale}std.nc')

In [ ]:
variables = ['2m_temperature',
 '10m_u_component_of_wind',
 '10m_v_component_of_wind',
 'mean_sea_level_pressure']
scale = 1
perturb_timestep = [0, 1]
with xr.open_dataset(f'testdata/stats/40yr_std_daily_4var.nc') as std: 
    for i in perturb_timestep:
        idx_time = input_data.time.isel(time=0).values.astype(np.int64)//21600
        normal_dist = np.random.normal(loc=0, scale=1, size=input_data[variables[0]].isel(time=i).shape)
        normal_dist = xr.DataArray(
            data=normal_dist,
            dims=('lat','lon'),
            coords={
                'lat': input_data.lat,
                'lon': input_data.lon
            }
        )
        perturbed = input_data.copy()
 
        for var in variables:
            perturbed.isel(time=i)[var] = input_data[var].isel(time=i) + scale * (normal_dist * std[var].isel(hour=idx_time))
            

    if 'perturbation' in list(perturbed.data_vars):
        perturbed = perturbed.drop_vars('perturbation')
perturbed



In [ ]:
xr.open_dataset(f'testdata/stats/40yr_std_daily_4var.nc').hour

# Scratch Codes

In [ ]:

# 5x5 numpy 배열 생성
data = np.arange(9).reshape(3,3)

# xarray DataArray 생성
da = xr.DataArray(
    data,
    dims=("latitude", "longitude"),  # 차원 이름 지정
    coords={
        "latitude": np.linspace(35, 39, 3),  # y 좌표
        "longitude": np.linspace(125, 129, 3) # x 좌표
    },
    name="example_data"  # DataArray의 이름
)

# 5x5 numpy 배열 생성
data = np.random.uniform(low=0, high=10, size=(3, 3))
data = np.round(data, decimals=0)

# xarray DataArray 생성
da2 = xr.DataArray(
    data,
    dims=("latitude", "longitude"),  # 차원 이름을 위도와 경도로 지정
    coords={
        "latitude": np.linspace(35, 39, 3),   # 위도 좌표 (35도에서 39도)
        "longitude": np.linspace(125, 129, 3) # 경도 좌표 (125도에서 129도)
    },
    name="temperature"  # DataArray의 이름을 temperature로 지정
)

data = np.random.uniform(low=0, high=10, size=(3, 3))
data = np.round(data, decimals=0)

# xarray DataArray 생성
da3 = xr.DataArray(
    data,
    dims=("latitude", "longitude"),  # 차원 이름을 위도와 경도로 지정
    coords={
        "latitude": np.linspace(35, 39, 3),   # 위도 좌표 (35도에서 39도)
        "longitude": np.linspace(125, 129, 3) # 경도 좌표 (125도에서 129도)
    },
    name="temperature"  # DataArray의 이름을 temperature로 지정
)

print(da)
print(da2)
print(da3)

da + 0.1 * (da2 * da3)

In [ ]:
# perturb_list = [
#        't2m', 
#        'v_wind', 
#        'u_wind', 
#        'sea_level_pressure'
#        ]

# import glob

# path = "/home/hiskim1/graphcast/testdata/stats/*/40yr_std_daily_*.nc"
# files = glob.glob(path)
# data = xr.open_mfdataset(files, combine='nested')
# data = data.rename({"v10": "10m_v_component_of_wind","u10": "10m_u_component_of_wind", "msl":"mean_sea_level_pressure"})
# data.to_netcdf('testdata/stats/40yr_std_daily_4var.nc')

data = xr.open_dataset('testdata/stats/40yr_std_daily_4var.nc')
data['10m_u_component_of_wind'].hour